In [7]:
import numpy as np
import pandas as pd
from pathlib import Path

In [8]:
path = Path('./data')

In [9]:
tweets = pd.read_csv(path/'tweets.csv', encoding = 'utf-8')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = tweets
df.head(5)

,tweetUrl,date,renderedContent,tweetId,userId,replyCount,retweetCount,likeCount,quoteCount,source,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/ShashiRajbhar6/status/1376...,2021-03-30 03:33:46+00:00,Support 👇\n\n#FarmersProtest,1.376739e+18,1.015970e+18,0,0,0,0,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN
1,https://twitter.com/kaursuk06272818/status/137...,2021-03-30 03:33:23+00:00,Supporting farmers means supporting our countr...,1.376739e+18,1.332937e+18,0,0,0,0,"<a href=""http://twitter.com/download/android"" ...",[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,NaN,NaN
2,https://twitter.com/kaursuk06272818/status/137...,2021-03-30 03:31:00+00:00,Support farmers if you are related to food #St...,1.376739e+18,1.332937e+18,0,0,0,0,"<a href=""http://twitter.com/download/android"" ...",[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,NaN,NaN
3,https://twitter.com/SukhdevSingh_/status/13767...,2021-03-30 03:30:45+00:00,#StopHateAgainstFarmers support #FarmersProtes...,1.376739e+18,1.308357e+18,0,1,3,0,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN
4,https://twitter.com/Davidmu66668113/status/137...,2021-03-30 03:30:30+00:00,"You hate farmers I hate you, \nif you love the...",1.376739e+18,1.357312e+18,0,0,1,0,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN


In [10]:
df = df[['tweetId', 'renderedContent']]

In [11]:
df.shape

(1015570, 2)

In [12]:
df = df.drop_duplicates(subset='renderedContent')

In [13]:
df.shape

(918639, 2)

In [14]:
import re
import demoji
import emoji
import operator
import functools  
from bs4 import BeautifulSoup
from itertools import chain
from autocorrect import spell
from textblob import TextBlob
from gensim.parsing.preprocessing import remove_stopwords
import string
combining_marks = '[\u0300-\u036F\u0483-\u0489\u0591-\u05BD\u05BF\u05C1\u05C2\u05C4\u05C5\u05C7\u0610-\u061A\u064B-\u065F\u0670\u06D6-\u06DC\u06DF-\u06E4\u06E7\u06E8\u06EA-\u06ED\u0711\u0730-\u074A\u07A6-\u07B0\u07EB-\u07F3\u07FD\u0816-\u0819\u081B-\u0823\u0825-\u0827\u0829-\u082D\u0859-\u085B\u08D3-\u08E1\u08E3-\u0903\u093A-\u093C\u093E-\u094F\u0951-\u0957\u0962\u0963\u0981-\u0983\u09BC\u09BE-\u09C4\u09C7\u09C8\u09CB-\u09CD\u09D7\u09E2\u09E3\u09FE\u0A01-\u0A03\u0A3C\u0A3E-\u0A42\u0A47\u0A48\u0A4B-\u0A4D\u0A51\u0A70\u0A71\u0A75\u0A81-\u0A83\u0ABC\u0ABE-\u0AC5\u0AC7-\u0AC9\u0ACB-\u0ACD\u0AE2\u0AE3\u0AFA-\u0AFF\u0B01-\u0B03\u0B3C\u0B3E-\u0B44\u0B47\u0B48\u0B4B-\u0B4D\u0B56\u0B57\u0B62\u0B63\u0B82\u0BBE-\u0BC2\u0BC6-\u0BC8\u0BCA-\u0BCD\u0BD7\u0C00-\u0C04\u0C3E-\u0C44\u0C46-\u0C48\u0C4A-\u0C4D\u0C55\u0C56\u0C62\u0C63\u0C81-\u0C83\u0CBC\u0CBE-\u0CC4\u0CC6-\u0CC8\u0CCA-\u0CCD\u0CD5\u0CD6\u0CE2\u0CE3\u0D00-\u0D03\u0D3B\u0D3C\u0D3E-\u0D44\u0D46-\u0D48\u0D4A-\u0D4D\u0D57\u0D62\u0D63\u0D82\u0D83\u0DCA\u0DCF-\u0DD4\u0DD6\u0DD8-\u0DDF\u0DF2\u0DF3\u0E31\u0E34-\u0E3A\u0E47-\u0E4E\u0EB1\u0EB4-\u0EBC\u0EC8-\u0ECD\u0F18\u0F19\u0F35\u0F37\u0F39\u0F3E\u0F3F\u0F71-\u0F84\u0F86\u0F87\u0F8D-\u0F97\u0F99-\u0FBC\u0FC6\u102B-\u103E\u1056-\u1059\u105E-\u1060\u1062-\u1064\u1067-\u106D\u1071-\u1074\u1082-\u108D\u108F\u109A-\u109D\u135D-\u135F\u1712-\u1714\u1732-\u1734\u1752\u1753\u1772\u1773\u17B4-\u17D3\u17DD\u180B-\u180D\u1885\u1886\u18A9\u1920-\u192B\u1930-\u193B\u1A17-\u1A1B\u1A55-\u1A5E\u1A60-\u1A7C\u1A7F\u1AB0-\u1ABE\u1B00-\u1B04\u1B34-\u1B44\u1B6B-\u1B73\u1B80-\u1B82\u1BA1-\u1BAD\u1BE6-\u1BF3\u1C24-\u1C37\u1CD0-\u1CD2\u1CD4-\u1CE8\u1CED\u1CF4\u1CF7-\u1CF9\u1DC0-\u1DF9\u1DFB-\u1DFF\u20D0-\u20F0\u2CEF-\u2CF1\u2D7F\u2DE0-\u2DFF\u302A-\u302F\u3099\u309A\uA66F-\uA672\uA674-\uA67D\uA69E\uA69F\uA6F0\uA6F1\uA802\uA806\uA80B\uA823-\uA827\uA880\uA881\uA8B4-\uA8C5\uA8E0-\uA8F1\uA8FF\uA926-\uA92D\uA947-\uA953\uA980-\uA983\uA9B3-\uA9C0\uA9E5\uAA29-\uAA36\uAA43\uAA4C\uAA4D\uAA7B-\uAA7D\uAAB0\uAAB2-\uAAB4\uAAB7\uAAB8\uAABE\uAABF\uAAC1\uAAEB-\uAAEF\uAAF5\uAAF6\uABE3-\uABEA\uABEC\uABED\uFB1E\uFE00-\uFE0F\uFE20-\uFE2F\U000101FD\U000102E0\U00010376-\U0001037A\U00010A01-\U00010A03\U00010A05\U00010A06\U00010A0C-\U00010A0F\U00010A38-\U00010A3A\U00010A3F\U00010AE5\U00010AE6\U00010D24-\U00010D27\U00010F46-\U00010F50\U00011000-\U00011002\U00011038-\U00011046\U0001107F-\U00011082\U000110B0-\U000110BA\U00011100-\U00011102\U00011127-\U00011134\U00011145\U00011146\U00011173\U00011180-\U00011182\U000111B3-\U000111C0\U000111C9-\U000111CC\U0001122C-\U00011237\U0001123E\U000112DF-\U000112EA\U00011300-\U00011303\U0001133B\U0001133C\U0001133E-\U00011344\U00011347\U00011348\U0001134B-\U0001134D\U00011357\U00011362\U00011363\U00011366-\U0001136C\U00011370-\U00011374\U00011435-\U00011446\U0001145E\U000114B0-\U000114C3\U000115AF-\U000115B5\U000115B8-\U000115C0\U000115DC\U000115DD\U00011630-\U00011640\U000116AB-\U000116B7\U0001171D-\U0001172B\U0001182C-\U0001183A\U000119D1-\U000119D7\U000119DA-\U000119E0\U000119E4\U00011A01-\U00011A0A\U00011A33-\U00011A39\U00011A3B-\U00011A3E\U00011A47\U00011A51-\U00011A5B\U00011A8A-\U00011A99\U00011C2F-\U00011C36\U00011C38-\U00011C3F\U00011C92-\U00011CA7\U00011CA9-\U00011CB6\U00011D31-\U00011D36\U00011D3A\U00011D3C\U00011D3D\U00011D3F-\U00011D45\U00011D47\U00011D8A-\U00011D8E\U00011D90\U00011D91\U00011D93-\U00011D97\U00011EF3-\U00011EF6\U00016AF0-\U00016AF4\U00016B30-\U00016B36\U00016F4F\U00016F51-\U00016F87\U00016F8F-\U00016F92\U0001BC9D\U0001BC9E\U0001D165-\U0001D169\U0001D16D-\U0001D172\U0001D17B-\U0001D182\U0001D185-\U0001D18B\U0001D1AA-\U0001D1AD\U0001D242-\U0001D244\U0001DA00-\U0001DA36\U0001DA3B-\U0001DA6C\U0001DA75\U0001DA84\U0001DA9B-\U0001DA9F\U0001DAA1-\U0001DAAF\U0001E000-\U0001E006\U0001E008-\U0001E018\U0001E01B-\U0001E021\U0001E023\U0001E024\U0001E026-\U0001E02A\U0001E130-\U0001E136\U0001E2EC-\U0001E2EF\U0001E8D0-\U0001E8D6\U0001E944-\U0001E94A\U000E0100-\U000E01EF]'

def cleanString(text):
    
    soup = BeautifulSoup(text, features="html.parser")
    text = str(soup.get_text())
    # to remove links that start with HTTP/HTTPS in the tweet
    text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', '', text, flags=re.MULTILINE) 
    text = re.sub('http[^\s]+','',text)
    # to remove other url links
    text = re.sub(r'[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', '', text, flags=re.MULTILINE) 

    
    text = re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE) #removing links
    text = re.sub(r'twitter?\.\S*', '', text, flags=re.MULTILINE)  #removing links
    text = re.sub(r'youtu?\.\S*', '', text, flags=re.MULTILINE)   #removing links
    
    #splitting hastags in words
#     text = re.sub(
#         r'#[a-zA-Z]\S*',lambda m: ' '.join(re.findall('[A-Z][^A-Z]*|[a-z][^A-Z]*', m.group().lstrip('#'))),
#         text,)
    
    text = re.sub("@[A-Za-z0-9_]+[:]*","", text)  #user mentions
#     text = re.sub("#(?:(?![A-Za-z])[\w]{}*)+".format(combining_marks),"", text)  #dropping hastags starting with non english characters

#     text = " ".join([spell(w) for w in text.split()])

    #replacing '\n' with space
    text = text.replace('\n', ' ')  
    
    #remove numbers
#     text = re.sub(r"\d", "", text)
    
    #removing punctuations
#     text = text.translate(str.maketrans('', '', string.punctuation))
    
    #converting emojis to plain text according to thier meanings
#     dic = demoji.findall(text)
#     em_split_emoji = emoji.get_emoji_regexp().split(text)
#     em_split_whitespace = [substr.split() for substr in em_split_emoji]
#     em_split = functools.reduce(operator.concat, em_split_whitespace)
#     reformed = [dic[word] if word in dic else word for word in em_split]
#     text = " ".join(reformed)
#     text = re.sub(r"_", " ", text)
    
    #removing stopwords
#     text = remove_stopwords(text)
#     text = text.lower()  #converting to all lower case 
    return text
df['renderedContent'] = df['renderedContent'].apply(cleanString)

In [15]:
df.head()

,tweetId,renderedContent
0,1.376739e+18,Support 👇 #FarmersProtest
1,1.376739e+18,Supporting farmers means supporting our countr...
2,1.376739e+18,Support farmers if you are related to food #St...
3,1.376739e+18,#StopHateAgainstFarmers support #FarmersProtes...
4,1.376739e+18,"You hate farmers I hate you, if you love them..."


In [16]:
df['split_tweet'] = [w.split() for w in df['renderedContent']]

In [17]:
df_greater_than_5 = df[df['split_tweet'].str.len()>5]

In [18]:
df_greater_than_5.shape

(839378, 3)

In [19]:
df.shape

(918639, 3)

In [20]:
df_greater_than_5.to_csv('data_for_stance_prediction.csv')

In [9]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from time import time 
from collections import defaultdict

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [10]:
import multiprocessing

In [6]:
# df = df.reset_index(drop=True)

In [7]:
# df = df[['tweetId', 'renderedContent']]

In [11]:
df.shape

(918639, 2)

In [12]:
df.head()

,tweetId,renderedContent
0,1.376739e+18,Support 👇 Farmers Protest
1,1.376739e+18,Supporting farmers means supporting our countr...
2,1.376739e+18,Support farmers if you are related to food Sto...
3,1.376739e+18,Stop Hate Against Farmers support Farmers Prot...
4,1.376739e+18,"You hate farmers I hate you, if you love them..."


In [91]:

# python -m spacy download en_core_web_sm

In [10]:
# import re
# def text_to_word_list(text):
#     text = str(text)
#     text = text.lower()

# #     # Clean the text
#     text = re.sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
#     text = re.sub(r"\+", " plus ", text)
#     text = re.sub(r",", " ", text)
#     text = re.sub(r"_", " ", text)
#     text = re.sub(r"\.", " ", text)
#     text = re.sub(r"!", " ! ", text)
#     text = re.sub(r"\?", " ? ", text)
#     text = re.sub(r"'", "", text)
#     text = re.sub(r":", " ", text)
#     text = re.sub(r"\s{2,}", " ", text)
# #     doc = nlp(text)
# #     text = " ".join([token.lemma_ for token in doc])
# #     text = text.lower()
# #     text = text.split()
#     return text 

In [11]:
# df.renderedContent = df.renderedContent.apply(text_to_word_list)

In [12]:
# df = df[df['renderedContent'].str.len()>4]

In [ ]:
# from googletrans import Translator
# translator = Translator()
# def translate(text):
#     try:
#         return Translator().translate(text)
#     except:
#         return text

# df.transalated_renderedContent = df.renderedContent.apply(translate)

In [ ]:
# df.head()

In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj = SentimentIntensityAnalyzer()
def get_vader_score(sent):
    # This polarity score is between -1 to 1
    sentiment_dict = sid_obj.polarity_scores(sent)
    return sentiment_dict['compound']
polarity = [get_vader_score(text) for text in df.renderedContent]

In [15]:
# polarity

In [16]:
df['sentiment_polarity'] = polarity

In [17]:
df.head()

,tweetId,renderedContent,sentiment_polarity
0,1.376739e+18,Support 👇 Farmers Protest,0.1779
1,1.376739e+18,Supporting farmers means supporting our countr...,-0.2732
2,1.376739e+18,Support farmers if you are related to food Sto...,-0.6369
3,1.376739e+18,Stop Hate Against Farmers support Farmers Prot...,-0.5719
4,1.376739e+18,"You hate farmers I hate you, if you love them...",-0.6476


In [24]:
df['sentiment'] = [-1 if x <=-0.05 else 1 if x >= 0.05 else 0 for x in df.sentiment_polarity]

In [25]:
df.head()

,tweetId,renderedContent,sentiment_polarity,sentiment
0,1.376739e+18,Support 👇 Farmers Protest,0.1779,1
1,1.376739e+18,Supporting farmers means supporting our countr...,-0.2732,-1
2,1.376739e+18,Support farmers if you are related to food Sto...,-0.6369,-1
3,1.376739e+18,Stop Hate Against Farmers support Farmers Prot...,-0.5719,-1
4,1.376739e+18,"You hate farmers I hate you, if you love them...",-0.6476,-1


In [26]:
textblob_supporting = df[df['sentiment'] == 1]
textblob_opposing = df[df['sentiment'] == -1]
textblob_neutral = df[df['sentiment'] == 0]

textblob_supporting['renderedContent'].to_csv('vader_supporting.txt', sep='\n')
textblob_opposing['renderedContent'].to_csv('vader_opposing.txt', sep='\n')
textblob_neutral['renderedContent'].to_csv('vader_neutral.txt', sep='\n')

In [16]:
textblob_opposing.shape

(176462, 4)

In [27]:
textblob_opposing.shape

(550109, 4)

In [17]:
textblob_supporting.shape

(347743, 4)

In [28]:
textblob_supporting.shape

(341411, 4)

In [18]:
textblob_neutral.shape

(394434, 4)

In [29]:
textblob_neutral.shape

(27119, 4)